In [75]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch

# extra imports
from collections import OrderedDict
from itertools import zip_longest
import datetime as dt
from datetime import timedelta

import glob
import fathon
from fathon import fathonUtils as fu


import matplotlib as mpl
from multiprocessing import Pool, freeze_support, Manager

import itertools
colormap = plt.cm.RdBu
plt.style.use(os.path.join(mpl.get_configdir(),'latexstyle.mplstyle'))
from collections import defaultdict
# import freqopttest.kernel as kernel

In [76]:
def linRangeByStep(start, end, step=1):
    """Array of linearly separated elements.
    
    Parameters
    ----------
    start : int
        Smallest element.
    end : int
        Biggest element.
    step : int
        Step between two consecutive elements (default : 1).
    Returns
    -------
    numpy ndarray
        Array of linearly separated elements.
    """
    return np.arange(start, end + 1, step, dtype=np.int64)

def linRangeByCount(start, end, count=-1):
    """Array of linearly separated elements.
    
    Parameters
    ----------
    start : int
        Smallest element.
    end : int
        Biggest element.
    count : int
        Number of elements (default : `end` - `start` + 1).
    Returns
    -------
    numpy ndarray
        Array of linearly separated elements.
    """
    if count == -1 or count > (end - start + 1):
        count = end - start + 1
        
    return np.linspace(start, end, count, endpoint=True, dtype=np.int64)

In [77]:
import os
os.listdir('/home/ak/.config/matplotlib/')

['latexstyle.mplstyle']

In [78]:
t71 = '/media/ak/T7/August11th2022Experiments/'
expInputFiles = os.path.join(t71,'ExperimentInputFiles' )
expOne = os.path.join(t71, 'ExperimentOne')
figuresDestination = '/home/ak/Documents/Papers/StylisedFactsPaper/figures/'
#n: Array of window's sizes used for the computation.
n =[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

In [93]:
symbols = ['FB1', 'JB1', 'XM1', 'TY1', 'TU1', 'RX1', 'YM1', 'US1', 'DU1']
#dollar-->YM1, US1 has not the right data
#tick -->
symbol = 'YM1'
symbol_path = os.path.join(expOne,str(symbol))
## bar
bar = 'tick'
bar_mfdfa_files =sorted([f for f in os.listdir(symbol_path) if str(bar) in f])
len(bar_mfdfa_files) # this is the number of files that have symbol and bar!

146

In [97]:
idx = 20
tickIdxMfdfaFileLoc = os.path.join(symbol_path, bar_mfdfa_files[idx]) # location of dictionary
# dictionary has the following keys:

dictf = pd.read_pickle(tickIdxMfdfaFileLoc)
dictf.keys()

dict_keys(['micro_price_change', 'arrival_rates', 'gk_vol', 'median_traded_volume', 'tick'])

In [98]:
#     Parameters
#     ----------
#     n : numpy ndarray
#         Array of window's sizes used for the computation.
#     tsVec : iterable
#         Time series used for the analysis.
#     F : numpy ndarray
#         Array containing the values of the fluctuations in each window.
#     listH : numpy ndarray
#         Array containing the values of the slope of the fit at each q-order.
#     qList : numpy ndarray
#         Array containing the values of the q-orders.

In [99]:
micro_variables = ['arrival_rates', 'gk_vol', 'median_traded_volume', 'micro_price_change'] # all the micro variables

gk_vol_dict = defaultdict(dict) # vol dictionary

#mfdfa details for the data above
arrival_rates_dict = defaultdict(dict)

median_traded_volume_dict = defaultdict(dict)
micro_price_change_dict = defaultdict(dict)

mfdfa_dict = defaultdict(dict)

tau_dict = defaultdict(dict)

alpha_dict = defaultdict(dict)

mfSpect_dict = defaultdict(dict)

list_H_intercept_dict = defaultdict(dict)

list_H_dict = defaultdict(dict)

n_F_dict = defaultdict(dict)


for idx in range(0, len(bar_mfdfa_files)):
    
    barIdxMfdfaFileLoc = os.path.join(symbol_path, bar_mfdfa_files[idx]) # location of file for symbol/bar combination
    pkl_dict = pd.read_pickle(barIdxMfdfaFileLoc) # read the pickle file
    keys = sorted(list(pd.read_pickle(barIdxMfdfaFileLoc).keys())) # get out all the keys
    
    bar = list(set(keys).difference(set(micro_variables)))[0] # get out the bar-choice
     
    gk_vol_dict[idx][bar] = pkl_dict['gk_vol'] # get the vol
    median_traded_volume_dict[idx][bar] = pkl_dict['median_traded_volume'] #get the volume
    arrival_rates_dict[idx][bar] = pkl_dict['arrival_rates'] #get arrival rates
    micro_price_change_dict[idx][bar] = pkl_dict['micro_price_change'] # get micro price change
    #---- start getting all the mfdfa variables ----#
    tau_dict[idx][bar] = pkl_dict[str(bar)]['tau'] # tau
    alpha_dict[idx][bar] = pkl_dict[str(bar)]['alpha'] # alpha
    mfSpect_dict[idx][bar] = pkl_dict[str(bar)]['mfSpect'] # mfSpect
    n_F_dict[idx][bar] = pkl_dict[str(bar)]['n_F'] # n_F
    list_H_dict[idx][bar] = pkl_dict[str(bar)]['list_H'] # list_H
    list_H_intercept_dict[idx][bar] = pkl_dict[str(bar)]['list_H_intercept'] # intercept
    n_F_dict[idx][bar] = pkl_dict[str(bar)]['n_F'] #n,F
    
    
    
    
    
                       
    


In [183]:
micro_price_change_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 'dollar', 'tick'])

In [100]:
# micro_price_change_dict

In [163]:
name='dollar'
d =micro_price_change_dict # just define a variable based on one dictionary
g = gk_vol_dict



In [102]:
# get all second level keys of all first level keys together (set comprehension)
sorted(list({k2 for v in d.values() for k2 in v}))

['dollar', 'tick', 'volume']

In [103]:
# get all first level keys

#list(d.keys())

In [114]:
# get all possible pairs of  (first, second) keys
tick_all_indices=[(k1,k2) for (k1, v) in d.items() for k2 in v if k2=='tick' ]
volume_all_indices=[(k1) for (k1, v) in d.items() for k2 in v if k2=='volume' ]
dollar_all_indices=[(k1) for (k1, v) in d.items() for k2 in v if k2=='dollar' ]

In [159]:
# tick_all_indices
# dollar_all_indices
# tick_all_indices

In [116]:
# sorted(list({k2 for v in d.values() for k2 in v}))

In [117]:

micro_price_change = dict()
mfSpect_bar_choice = dict()
alpha_bar_choice = dict()
gk_vol = dict()


for f in dollar_all_indices: 
    
    micro_price_change[f] = d[f][str(bar)]
    mfSpect_bar_choice[f] = mfSpect_dict[f][str(bar)] 
    alpha_bar_choice[f] = alpha_dict[f][str(bar)] 
    gk_vol[f] = gk_vol_dict[f][str(bar)]

In [118]:
pd.DataFrame.from_dict(gk_vol).median(axis=1).describe()

count    3.319240e+05
mean     2.401036e-09
std      8.679750e-08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.060986e-06
dtype: float64

In [186]:
mpc_df =pd.DataFrame.from_dict(micro_price_change).fillna(0)
mpc_df.shape

(331924, 79)

In [185]:
mfSpect_bar_choice.keys()

dict_keys([0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 16, 17, 18, 19, 23, 24, 25, 26, 27, 28, 36, 37, 38, 45, 49, 50, 51, 52, 53, 54, 55, 57, 60, 61, 62, 66, 69, 70, 72, 73, 74, 75, 76, 77, 78, 82, 83, 84, 85, 86, 89, 90, 91, 92, 93, 94, 95, 102, 103, 104, 111, 119, 120, 121, 125, 126, 127, 132, 133, 134, 138, 139, 140, 141, 142, 143, 144, 145])

In [148]:
std_df =pd.DataFrame()


In [150]:
std= mpc_df.std(axis=0)
std_df['std'] = std
first_regime = np.quantile(std, 0.33) # thresholds based on the definition of a median
second_regime = np.quantile(std, 0.66)

In [140]:
print(first_regime, second_regime)

0.007762186968310944 0.01097705350998804


In [157]:
std_df
std_df['LowRegime'] = std_df['std'].apply(lambda x: '1' if x <= first_regime else '0')
std_df['HiRegime'] = std_df['std'].apply(lambda x: '1' if x >= second_regime else '0')
std_df['MidRegime'] = std_df['std'].apply(lambda x: '1' if first_regime<x < second_regime else '0')




In [176]:
std_df.shape

(79, 4)

In [170]:
std_df
low_regime_indices = std_df.index[std_df['LowRegime'] == '1'].tolist()
hi_regime_indices = std_df.index[std_df['HiRegime'] == '1'].tolist()
mid_regime_indices = std_df.index[std_df['MidRegime'] == '1'].tolist()

In [178]:
len(hi_regime_indices)+len(low_regime_indices)+len(mid_regime_indices) 

# these are all dates, so this should match to the shape of the mpc dataframe in terms of dates!

79

In [38]:
# for f in tick_all_indices:
#     print(d[f]['tick'])
mf_spect_df = pd.DataFrame.from_dict(mfSpect_bar_choice).dropna(0)
alpha_df = pd.DataFrame.from_dict(alpha_bar_choice).dropna(0)

print(mf_spect_df.shape, alpha_df.shape)
#rows x columns

(38, 98) (38, 98)


In [18]:
df = pd.DataFrame.from_dict(micro_price_change).dropna()
median_all_pct_change= pd.DataFrame(df.median(axis=0))
median_all_pct_change.shape
len(list(df.columns))

98

In [142]:
gk_vol_df = pd.DataFrame.from_dict(gk_vol)
mean = pd.DataFrame.from_dict(gk_vol).median(axis=0)
first_regime = np.quantile(mean, 0.33) # thresholds based on the definition of a median
second_regime = np.quantile(mean, 0.66)

In [20]:
# df['regime'], b = pd.qcut(df['median'].values, q = [0, 0.33, 0.66, 1], duplicates = 'drop',retbins = True)
# # c, b = pd.qcut(df['median'].values, q = [0, 0.25, 0.5, 0.75, 1], duplicates = 'drop',retbins = True)

In [141]:
mean.describe()

count    97.000000
mean      0.000050
std       0.000025
min       0.000000
25%       0.000057
50%       0.000061
75%       0.000064
max       0.000076
dtype: float64

In [61]:
regime_df = pd.DataFrame(regime_dict)
regimes_count = dict()
for i in range(0,regime_df.shape[1]):
    regimes_count[i] = (regime_df.iloc[i].value_counts())


In [62]:
regime_counts_df = pd.DataFrame(regimes_count)

In [63]:

regime_counts_df # now you have how much time the price change has been on each regime!

""


In [42]:
a= (micro_price_change_dict[1]['dollar'])

In [43]:
pydfa = fathon.DFA(a)

In [27]:
winSizes = fu.linRangeByStep(10, 2000)
revSeg = True
polOrd = 3

In [28]:
%%time
n, F = pydfa.computeFlucVec(winSizes, revSeg=revSeg, polOrd=polOrd)

CPU times: user 7.51 s, sys: 57.9 ms, total: 7.57 s
Wall time: 494 ms


In [29]:
len(n)

1991